# H2
**Goal:** The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and model management. 

---

## Q1. Install MLflow  
To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
!which python

/opt/conda/envs/mod2/bin/python


In [2]:
!mlflow --version

mlflow, version 2.22.0


### What's the version that you have?  
Version 2.22.0

---

## Q2. Download and preprocess the data  
We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [3]:
!python preprocess_data.py --help

Usage: preprocess_data.py [OPTIONS]

Options:
  --raw_data_path TEXT  Location where the raw NYC taxi trip data was saved
  --dest_path TEXT      Location where the resulting files will be saved
  --help                Show this message and exit.


In [4]:
!python preprocess_data.py --raw_data_path ./data/raw --dest_path ./data/preprocessed

In [5]:
!ls ./data/preprocessed
!echo "Numero di file:" $(ls ./data/preprocessed | wc -l)

dv.pkl	test.pkl  train.pkl  val.pkl
Numero di file: 4


### A2. How many files were saved to `OUTPUT_FOLDER`?  
4

---

## Q3. Train a model with autolog  
We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter?

* 2
* 4
* 8
* 10

In [6]:
import mlflow

MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'
MLFLOW_EXPERIMENT_ID = 'hw2-experiment'

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_ID)

2025/05/27 11:05:50 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/05/27 11:05:50 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/mlruns/1', creation_time=1748343951432, experiment_id='1', last_update_time=1748343951432, lifecycle_stage='active', name='hw2-experiment', tags={}>

In [7]:
!python train.py --help

Usage: train.py [OPTIONS]

Options:
  --data_path TEXT  Location where the processed NYC taxi trip data was saved
  --help            Show this message and exit.


In [8]:
!python train.py --data_path ./data/preprocessed

In [9]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/mlruns/1', creation_time=1748343951432, experiment_id='1', last_update_time=1748343951432, lifecycle_stage='active', name='hw2-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/mlruns/0', creation_time=1748343951427, experiment_id='0', last_update_time=1748343951427, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',                     
    filter_string='',                       
    run_view_type=ViewType.ACTIVE_ONLY,     
    max_results=5,                         
    order_by=['attribute.start_time ASC']     
)

In [11]:
for run in runs:
    print(f'run id: {run.info.run_id}, rmse: {run.data.metrics['training_root_mean_squared_error']:.4f}')

run id: 25f6053c64e743a09d9ee58b6a0bf597, rmse: 5.2041


In [12]:
# first (i.e oldest)
f'min_sample_split {runs[0].data.params["min_samples_split"]}'

'min_sample_split 2'

### A3. What is the value of the `min_samples_split` parameter?  
2

---

## Q4. Launch the tracking server locally
Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

In [13]:
"""
    mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts
"""

'\n    mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts\n'

### A4. In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?  
default-artifact-root

---

## Q5. Tune model hyperparameters
Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336

In [14]:
!python hpo.py --help

2025/05/27 11:06:44 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
Usage: hpo.py [OPTIONS]

Options:
  --data_path TEXT      Location where the processed NYC taxi trip data was
                        saved
  --num_trials INTEGER  The number of parameter evaluations for the optimizer
                        to explore
  --help                Show this message and exit.


In [15]:
!python hpo.py --data_path ./data/preprocessed

🏃 View run traveling-jay-583 at: http://127.0.0.1:5000/#/experiments/2/runs/c0bc675ce99246d88f542d69f6471083

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run intrigued-chimp-329 at: http://127.0.0.1:5000/#/experiments/2/runs/b80823cb44b842409d73daf934f1745b

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run rare-snail-51 at: http://127.0.0.1:5000/#/experiments/2/runs/8f10a925e1714fd295c3324c76d51e45

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run peaceful-wolf-61 at: http://127.0.0.1:5000/#/experiments/2/runs/e77e55634d0f4237bca1016068ecde77

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run auspicious-mare-941 at: http://127.0.0.1:5000/#/experiments/2/runs/200e524b2c8840b580ab0381725799f5

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run unique-snake-760 at: http://127.0.0

In [16]:
runs = client.search_runs(
    experiment_ids='2',                     
    filter_string='',                       
    run_view_type=ViewType.ACTIVE_ONLY,     
    max_results=5,                         
    order_by=['metrics.rmse ASC'] 
)

In [17]:
# top 5
for run in runs:
    print(f'run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}')

run id: 06fd57d8f6014a66b05d43fd406d61ac, rmse: 5.3354
run id: e9e9cf669a1f4c098ca3b9649e8643ad, rmse: 5.3547
run id: 0e908f1b238b4bedb4bd8307da35c05a, rmse: 5.3550
run id: e77e55634d0f4237bca1016068ecde77, rmse: 5.3575
run id: d73beb9b4ca049d998e180d11f9832cb, rmse: 5.3634


### A5. What's the best validation RMSE that you got?  
5.335

---

## Q6. Promote the best model to the model registry
The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [18]:
!python register_model.py --help

Usage: register_model.py [OPTIONS]

Options:
  --data_path TEXT  Location where the processed NYC taxi trip data was saved
  --top_n INTEGER   Number of top models that need to be evaluated to decide
                    which one to promote
  --help            Show this message and exit.


In [19]:
!python register_model.py --data_path ./data/preprocessed

Searching for top 5 models from random-forest-hyperopt...
Found 5 runs from hyperparameter optimization
Training model 1/5...
2025/05/27 11:07:59 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2025/05/27 11:08:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Logged model with test_rmse: 5.567408012462019
🏃 View run invincible-calf-413 at: http://127.0.0.1:5000/#/experiments/3/runs/0f60aa4678d941ebaa30c931dd2ea8a6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
Training model 2/5...
2025/05/27 11:08:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Logged model with test_rmse: 5.58531221803063
🏃 View run bustling-cow-711 at: http://127.0.0.1:5000/#

In [20]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/3', creation_time=1748344079856, experiment_id='3', last_update_time=1748344079856, lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/artifacts/2', creation_time=1748344004607, experiment_id='2', last_update_time=1748344004607, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/mlruns/1', creation_time=1748343951432, experiment_id='1', last_update_time=1748343951432, lifecycle_stage='active', name='hw2-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/mlruns/0', creation_time=1748343951427, experiment_id='0', last_update_time=1748343951427, lifecycle_stage='active', name='Default', tags={}>]

In [21]:
best_test_run = client.search_runs(
    experiment_ids='3',                     
    order_by=['metrics.test_rmse ASC'] 
)[0]

In [22]:
f'Best model RMSE: {best_test_run.data.metrics['test_rmse']:.3f}'

'Best model RMSE: 5.567'

### A6. What is the test RMSE of the best model?  
5.567